# Face landmark detection with Dlib
使用 Dlib 預測 68 facial landmarks，用來作為 FFHQ face alignment 的輸入

In [4]:
from pathlib import Path
from random import sample
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import cv2
import dlib
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageDraw
from IPython.display import display

In [5]:
LANDMARK_PREDICTOR = Path('~/models/dlib/shape_predictor_68_face_landmarks.dat').expanduser()
DATA_PATHS = list(Path('deepfashion_1024x1024/').glob('*.jpg'))
len(DATA_PATHS)

27753

In [6]:
def rect_to_bb(rect):
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
	coords = np.zeros((68, 2), dtype=dtype)
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
        
	# return the list of (x, y)-coordinates
	return coords

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(str(LANDMARK_PREDICTOR))

In [ ]:
%%time
SHOW = False
RETURN_SCORES = False
num = 0

def detect_face_and_landmarks(p, return_scores=False):
    boxes, shapes = [], []
    img = cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB)
    # rects = detector(img, 1)
    rects, scores, idx = detector.run(img, 1)
    
    for (i, rect) in enumerate(rects):
        shape = predictor(img, rect)
        shapes.append(shape_to_np(shape))
        boxes.append(rect_to_bb(rect))
        
    if return_scores:
        return boxes, shapes, scores, idx
    
    return boxes, shapes
        
with ThreadPoolExecutor() as executor:
    func = partial(detect_face_and_landmarks, return_scores=RETURN_SCORES)
    results = executor.map(func, sample(DATA_PATHS, 100))
    
    for result in results:
        if RETURN_SCORES:
            boxes, shapes, scores, idx = result
        else:
            boxes, shapes = result
        
        for box, shape in zip(boxes, shapes):
            x, y, w, h = box
    
            if SHOW:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(img, "Face #{}".format(i + 1), (x - 10, y - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                for (x, y) in shape:
                    cv2.circle(img, (x, y), 1, (0, 0, 255), -1)
    if SHOW:
        display(Image.fromarray(img))

### Save to json file (FFHQ format)

In [17]:
outputs = {}
num = 0
for p in tqdm(DATA_PATHS):
    num += 1
    output = {'in_the_wild': {}}
    output['in_the_wild']['file_path'] = str(p)
    img = cv2.cvtColor(cv2.imread(str(p)), cv2.COLOR_BGR2RGB)
    rects = detector(img, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(img, rect)
        shape = shape_to_np(shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = rect_to_bb(rect)
        output['in_the_wild']['box'] = [x, y, w, h]
        output['in_the_wild']['face_landmarks'] = shape.tolist()
        #cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # show the face number
        #cv2.putText(img, "Face #{}".format(i + 1), (x - 10, y - 10),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        #for (x, y) in shape:
        #    cv2.circle(img, (x, y), 1, (0, 0, 255), -1)
    outputs[str(num)] = output

100%|██████████| 32796/32796 [5:17:07<00:00,  1.72it/s]  


In [1]:
import json
json.dump(outputs, open('df_landmarks.json', 'w'))